In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd

In [2]:
# Define the tickers and date range
tickers = ['TSLA', 'BND', 'SPY']  # Example tickers
start_date = '2015-01-01'        # Start date
end_date = '2025-01-31'          # End date

In [3]:
data = yf.download(tickers, start=start_date, end=end_date)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed


In [6]:
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

In [7]:
# Print the columns to understand the structure
print("Train Data Columns:")
print(train.columns)

Train Data Columns:
MultiIndex([( 'Close',  'BND'),
            ( 'Close',  'SPY'),
            ( 'Close', 'TSLA'),
            (  'High',  'BND'),
            (  'High',  'SPY'),
            (  'High', 'TSLA'),
            (   'Low',  'BND'),
            (   'Low',  'SPY'),
            (   'Low', 'TSLA'),
            (  'Open',  'BND'),
            (  'Open',  'SPY'),
            (  'Open', 'TSLA'),
            ('Volume',  'BND'),
            ('Volume',  'SPY'),
            ('Volume', 'TSLA')],
           names=['Price', 'Ticker'])


In [8]:
from statsmodels.tsa.arima.model import ARIMA

# Fit ARIMA model to TSLA's Adjusted Close prices
tsla_adj_close = train['Adj Close']['TSLA']
model = ARIMA(tsla_adj_close, order=(5, 1, 0))  # Adjust parameters as needed
model_fit = model.fit()

KeyError: 'Adj Close'

In [9]:
try:
    # Access Close prices for TSLA
    tsla_close = train['Close']['TSLA']
    model = ARIMA(tsla_close, order=(5, 1, 0))  # Adjust parameters as needed
    model_fit = model.fit()
    
    # Print model summary
    print(model_fit.summary())
except KeyError as e:
    print(f"KeyError: {e}. Please check the column names.")
except NameError as e:
    print(f"NameError: {e}. Ensure all previous code has run successfully.")

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:                   TSLA   No. Observations:                 2028
Model:                 ARIMA(5, 1, 0)   Log Likelihood               -6334.885
Date:                Mon, 03 Mar 2025   AIC                          12681.771
Time:                        15:04:53   BIC                          12715.457
Sample:                             0   HQIC                         12694.131
                               - 2028                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0376      0.012     -3.103      0.002      -0.061      -0.014
ar.L2          0.0154      0.010      1.496      0.135      -0.005       0.036
ar.L3      -1.106e-05      0.013     -0.001      0.9

In [10]:
forecast = model_fit.forecast(steps=len(test))

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [14]:
import sklearn
print(sklearn.__version__)

1.6.1


In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming `forecast` is already defined and contains your model's predictions
try:
    # Access Close prices for TSLA in the test set
    tsla_close_test = test['Close']['TSLA']
    
    # Calculate Mean Absolute Error
    mae = mean_absolute_error(tsla_close_test, forecast)
    
    # Calculate Mean Squared Error and then RMSE manually
    mse = mean_squared_error(tsla_close_test, forecast)
    rmse = mse ** 0.5  # Calculate RMSE manually

    # Print the metrics
    print(f'Mean Absolute Error: {mae}')
    print(f'Root Mean Squared Error: {rmse}')
except KeyError as e:
    print(f"KeyError: {e}. Please check the column names.")
except NameError as e:
    print(f"NameError: {e}. Ensure all previous code has run successfully.")

Mean Absolute Error: 89.8022800124448
Root Mean Squared Error: 110.9893220457153
